## Data Loading

In [ ]:
import requests
import io

import pandas as pd

DE_TEST_FILE  = 'deu_test.json'
DE_TRAIN_FILE = 'deu_training.json'
FR_TEST_FILE  = 'fra_test.json'
FR_TRAIN_FILE = 'fra_training.json'
ES_TEST_FILE  = 'spa_test.json'
ES_TRAIN_FILE = 'spa_training.json'

def read_json_from_git(fname):
    url = f'https://raw.githubusercontent.com/wortcook/LLMs-from-scratch/refs/heads/main/data/{fname}'
    s = requests.get(url).content
    
    return pd.read_json(io.StringIO(s.decode('utf-8')))

de_test  = read_json_from_git(DE_TEST_FILE)
print(de_test.head())


#tokenize
def tokenize(sentence):
    return sentence.split()

vocab = []

#def add to vocab
def add_to_vocab(word):
    if word not in vocab:
        vocab.append(word)
    return vocab.index(word)

#iterating over the rows of the dataframe
for index, row in de_test.iterrows():
    row_tokens = tokenize(row['source'])
    
    token_ids = []
    
    for token in row_tokens:
        token_ids.append(add_to_vocab(token))
        
    de_test.at[index, 'source_tokens'] = row_tokens
    de_test.at[index, 'source_token_ids'] = token_ids
    
print(de_test.head())



  source_language target_lang  \
0              en          de   
1              en          de   
2              en          de   
3              en          de   
4              en          de   

                                                data  
0  {'source': 'I think that she won't come.', 'ta...  
1  {'source': 'I often have ear infections.', 'ta...  
2  {'source': 'You're correct.', 'target': 'Sie h...  
3  {'source': 'Your opinion is quite different fr...  
4  {'source': 'A lot of new buildings have gone u...  
